## 8 Case Study II: Document Classification
In this case, we have two categories of emails, in which one category is about hockey and the other is about baseball. The data is in the folder **classification**.

In [32]:
# Import module and define global variables
import re
import os
import math
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

PUNCTUATION = "~!@#$%^&*()_+`{}|\[\]\:\";\-\\\='<>?,./"
STEMMER = PorterStemmer()

In [2]:
# test cell
from sklearn.naive_bayes import GaussianNB

### a) Firstly preprocess the documents into numerical data (Record data). The preprocessing guidelines can be found in the **introduction slides (SMO)**, consider using tf-idf (referring to Question 1).

In [4]:
def is_stop_word(word):
    return word in  stopwords.words('english')

def stem_word(word):
    return STEMMER.stem(word)

def filter_line(line):
    return re.sub(r'[{}]+'.format(PUNCTUATION), ' ', line.strip('\n')).strip().lower()

def walk_load_files(path):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
             with open(os.path.join(dirname, filename),errors='ignore') as f:
                yield filename,f

def walk_filter_lines(path):
    for _,f in walk_load_files(path):
        for line in f.readlines():
            yield filter_line(line)

def build_vocabulary_base(base_dir='/kaggle/input/emailtexts/data/emailtexts'):
    vocabulary_base = {}
    for line in walk_filter_lines(base_dir):
        for i in line.split():
            if is_stop_word(i):
                continue
            word = stem_word(i)
            vocabulary_base[word] = vocabulary_base.get(word,0) + 1
    return vocabulary_base

In [5]:
# compute the tf-idf value
"""
Follow the steps below:
- Calculate the tf value of each word
- Calculate the idf value of each word
- Compute the tf-idf value
"""
def cal_file_tf(vocabulary_base,filehandler):
    res = {}
    all_words = 0
    # init res dict
    for word,_ in vocabulary_base.items():
        res[word] = 0
    for line in filehandler:
        for word in filter_line(line).split():
            all_words += 1
            if is_stop_word(word):
                continue
            word = stem_word(word)
            res[word] += 1
    # compute tf
    for word,count in res.items():
        res[word] = count/all_words
    return res

def tf(vocabulary_base,base_dir):
    tf_dic = {}
    for filename,f in walk_load_files(base_dir):
        tf_dic[filename] = cal_file_tf(vocabulary_base,f)
    # tf_df = pd.DataFrame(tf_dic)
    return tf_dic

def idf(vocabulary_base,tf_dic):
    idf_dic = {}
    for word,_ in vocabulary_base.items():
        count = 0
        file_num = 0
        for _,tf_v in tf_dic.items():
            file_num += 1
            if tf_v[word] == 0:
                continue
            count += 1
        if not count:print(word)
        idf_dic[word] = math.log((1+file_num)/count)
    return idf_dic

def tf_idf(vocabulary_base,base_dir='/kaggle/input/emailtexts/data/emailtexts'):
    tf_dic = tf(vocabulary_base,base_dir)
    idf_dic = idf(vocabulary_base,tf_dic)
    tf_idf_dic = {}
    for filename,tf_vec in tf_dic.items():
        value = {}
        for word,_ in vocabulary_base.items():
            value[word] = tf_vec[word]*idf_dic[word]
        tf_idf_dic[filename] = value
    tfidf_df = pd.DataFrame(tf_idf_dic).T
    return tfidf_df,tf_idf_dic

In [27]:
# use tf-idf to preprocess the documents into numerical data.
BASE_DIR4 = './data/classification'
# remove mac os index file .DS_Store
os.system("rm ./data/classification/.DS_Store")
os.system("rm ./data/classification/baseball/.DS_Store")
os.system("rm ./data/classification/hockey/.DS_Store")
tc_vb = build_vocabulary_base(base_dir=BASE_DIR4)
tc_tfidf_df,tc_tf_idf_dic = tf_idf(vocabulary_base=tc_vb,base_dir=BASE_DIR4)
tc_tfidf_df

rm: ./data/classification/baseball/.DS_Store: No such file or directory
rm: ./data/classification/hockey/.DS_Store: No such file or directory


,spl2,po,cwru,edu,sam,lubchanski,subject,joe,robbi,stadium,...,helsingfor,ifk,1897,exager,proopos,heintz,gradual,scandinavia,deduct,variant
102700,0.072688,0.038134,0.051662,0.007540,0.040465,0.072688,4.809941e-06,0.041958,0.059740,0.067337,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104403,0.000000,0.000000,0.000000,0.001713,0.000000,0.000000,2.731733e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104631,0.000000,0.000000,0.000000,0.001538,0.000000,0.000000,8.172990e-07,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
105989,0.000000,0.000000,0.000000,0.001843,0.000000,0.000000,2.939409e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
104890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.116975e-05,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.616107e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
54507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.312373e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
53722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.961884e-07,0.005771,0.000000,0.000000,...,0.007498,0.007498,0.007498,0.007498,0.007498,0.007498,0.014997,0.007498,0.007498,0.007498
54163,0.000000,0.000000,0.000000,0.004951,0.000000,0.000000,2.631617e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### b) Use SVMs to classify the documents and test the classification results with 5-fold cross validation. You should report the precision, recall, and F1-measure of each fold and the average values. (Recommend LIBSVM to implement SVMs. You can refer to the tutorial slides in evaluating the results.)

In [29]:
# add label to tf-idf dataframe,0 for baseball and 1 for hockey
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score,recall_score,f1_score


def add_label(df,label_dir):
    label = {}
    for dirname, _, filenames in os.walk(label_dir):
        for filename in filenames:
            label[filename] = dirname.split('/')[-1]
    for i in df.index:
        if label[i] == 'baseball':
            df.loc[i, 'LABEL'] = 0
        else:
            df.loc[i, 'LABEL'] = 1
    df["LABEL"] = df["LABEL"].astype("int")
    return df

email_dataset = add_label(tc_tfidf_df,BASE_DIR4)
email_dataset

,spl2,po,cwru,edu,sam,lubchanski,subject,joe,robbi,stadium,...,ifk,1897,exager,proopos,heintz,gradual,scandinavia,deduct,variant,LABEL
102700,0.072688,0.038134,0.051662,0.007540,0.040465,0.072688,4.809941e-06,0.041958,0.059740,0.067337,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
104403,0.000000,0.000000,0.000000,0.001713,0.000000,0.000000,2.731733e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
104631,0.000000,0.000000,0.000000,0.001538,0.000000,0.000000,8.172990e-07,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
105989,0.000000,0.000000,0.000000,0.001843,0.000000,0.000000,2.939409e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
104890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.116975e-05,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.616107e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
54507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.312373e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
53722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.961884e-07,0.005771,0.000000,0.000000,...,0.007498,0.007498,0.007498,0.007498,0.007498,0.014997,0.007498,0.007498,0.007498,1
54163,0.000000,0.000000,0.000000,0.004951,0.000000,0.000000,2.631617e-06,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [30]:
# use Naive Bayes to classification the email.
def train_svm_model(data):
    models = []
    tests = []
    kf = KFold(n_splits=5,shuffle=True)
    for train_index,test_index in tqdm(kf.split(data)):
        gnb = SVC()
        train = data.iloc[train_index]
        test = data.iloc[test_index]
        models.append(gnb.fit(train.iloc[:,:-1], train["LABEL"]))
        tests.append(test)
    return models,tests

models,tests = train_svm_model(email_dataset)

5it [01:49, 21.83s/it]


In [33]:
# report result for the classification
# precision, recall, and F1-measure of each fold and the average values.

def report_result(models,tests):
    ps,rs,fs = [],[],[]
    for i in range(len(models)):
        y_true = tests[i]["LABEL"]
        y_pred = models[i].predict(tests[i].iloc[:,:-1])
        ps.append(precision_score(y_true,y_pred))
        rs.append(recall_score(y_true,y_pred))
        fs.append(f1_score(y_true,y_pred))
        print(f"Fold {i}: precision {ps[-1]}, recall {rs[-1]}, F1-measure {fs[-1]}.")
    print(f"Avg: precision {np.mean(ps)}, recall {np.mean(rs)}, F1-measure {np.mean(fs)}.")

report_result(models,tests)

Fold 0: precision 0.9842105263157894, recall 0.9689119170984456, F1-measure 0.9765013054830286.
Fold 1: precision 1.0, recall 0.915, F1-measure 0.9556135770234987.
Fold 2: precision 0.9856459330143541, recall 0.958139534883721, F1-measure 0.9716981132075472.
Fold 3: precision 0.9777777777777777, recall 0.946236559139785, F1-measure 0.9617486338797814.
Fold 4: precision 0.9842931937172775, recall 0.9353233830845771, F1-measure 0.9591836734693876.
Avg: precision 0.9863854861650398, recall 0.9447222788413058, F1-measure 0.9649490606126486.


### c) **Bonus (5 extra points).** Implement Sequential Minimal Optimization (SMO) by following the introduction slides.

In [34]:
# 本代码部分内容参考自 https://zhuanlan.zhihu.com/p/157858995
# 本代码部分内容参考自 

def decision_function_output(i):
    global m,b
    return np.sum([alpha[j] * target[j] * kernel(point[j], point[i]) for j in range(m)]) - b

def svm_output(alphas, target, kernel, X_train, x_test, b):
    result = (alphas * target) @ kernel(X_train, x_test) - b
    return result

def linear_kernel(x,y,b=1):
    #线性核函数
    result = x @ y.T + b
    return result

def gaussian_kernel(x,y, sigma=1):
    #高斯核函数
    if np.ndim(x) == 1 and np.ndim(y) == 1:
        result = np.exp(-(np.linalg.norm(x-y,2))**2/(2*sigma**2))
    elif(np.ndim(x)>1 and np.ndim(y) == 1) or (np.ndim(x) == 1 and np.ndim(y)>1):
        result = np.exp(-(np.linalg.norm(x-y, 2, axis=1)**2)/(2*sigma**2))
    elif np.ndim(x) > 1 and np.ndim(y) > 1 :
        result = np.exp(-(np.linalg.norm(x[:, np.newaxis]- y[np.newaxis, :], 2, axis = 2) ** 2)/(2*sigma**2))
    return result

def get_error(i1):
    if 0< alpha[i1] < C:
        return errors[i1]
    else:
        return decision_function_output(i1) - target[i1]

C = 20
b = 0
target = email_dataset.iloc[500:1500,:]["LABEL"]
target[target == 0] = -1
target = np.array(target)
point = np.array(email_dataset.iloc[500:1500,400:1400])
m,n = np.shape(point)
tol = 0.01
eps = 0.01
alpha = [0 for _ in range(len(point))]
kernel = linear_kernel
errors = svm_output(alpha, target, kernel, point, point, b) - target


def takeStep(i1,i2):
    global b
    if i1 == i2:
        return 0
    alph1 = alpha[i1]
    y1 = target[i1]
    E1 = get_error(i1)
    alph2 = alpha[i2]
    y2 = target[i2]
    E2 = get_error(i2)
    s = y1*y2
    # Compute L, H
    if(y1 != y2):   
        L = max(0, alph2 - alph1)
        H = min(C, C + alph2 - alph1)
    elif (y1 == y2):
        L = max(0, alph1+alph2 - C)
        H = min(C, alph1 + alph2)
    if L == H:
        return 0
    k11 = kernel(point[i1],point[i1])
    k12 = kernel(point[i1],point[i2])
    k22 = kernel(point[i2],point[i2])
    eta = 2*k12-k11-k22
    if eta < 0:
        a2 = alph2 - y2*(E1-E2)/eta
        if a2 < L:
            a2 = L
        elif a2 > H:
            a2 = H
    else:
        f1 = y1*(E1 + b) - alph1*k11 - s*alph2*k12
        f2 = y2 * (E2 + b) - s* alph1 * k12 - alph2 * k22
        L1 = alph1 + s*(alph2 - L)
        H1 = alph1 + s*(alph2 - H)
        Lobj = L1 * f1 + L * f2 + 0.5 * (L1 ** 2) * k11 + 0.5 * (L**2) * k22 + s * L * L1 * k12
        Hobj = H1 * f1 + H * f2 + 0.5 * (H1**2) * k11 + 0.5 * (H**2) * k22 + s * H * H1 * k12

        if Lobj < Hobj - eps:
            a2 = H
        elif Lobj > Hobj + eps:
            a2 = L
        else:
            a2 = alph2

    if a2 <1e-8:
        a2 = 0.0
    elif a2 > (C - 1e-8):
        a2 = C

    if (np.abs(a2 - alph2) < eps * (a2 + alph2 + eps)):
        return 0

    a1 = alph1 + s * (alph2 - a2)

    #更新 bias b的值 Equation (J20)
    b1 = E1 + y1*(a1 - alph1) * k11 + y2 * (a2 - alph2) * k12 + b
    #equation (J21)
    b2 = E2 + y1*(a1 - alph1) * k12 + y2 * (a2 - alph2) * k22 + b
    # Set new threshoold based on if a1 or a2 is bound by L and/or H
    if 0 < a1 and a1 < C:
        b_new =b1
    elif 0 < a2 and a2 < C:
        b_new = b2
    #Average thresholds if both are bound
    else:
        b_new = (b1 + b2) * 0.5
    #update model threshold
    b = b_new

    #优化完了，更新差值矩阵的对应值
    #同时更新差值矩阵其它值
    errors[i1] = 0
    errors[i2] = 0
    #更新差值 Equation (12)
    for i in range(m):
        if 0 < alpha[i] < C:
            errors[i] += y1*(a1 - alph1)*kernel(point[i1],point[i]) + y2*(a2 - alph2)*kernel(point[i2], point[i]) + b - b_new
    alpha[i1] = a1
    alpha[i2] = a2
    return 1



def examineExample(i2):
    y2 = target[i2]
    alph2 = alpha[i2]
    E2 = get_error(i2)
    r2 = E2*y2
    if (r2 < -tol and alph2 < C) or (r2 > tol and alph2 > 0):
        if (len(alpha) - alpha.count(0) - alpha.count(C) > 1):
            if errors[i2] > 0:
                i1 = np.argmin(errors)
            elif errors[i2] <= 0:
                i1 = np.argmax(errors)
            if takeStep(i1,i2):
                return 1
        rand = random.randint(0,len(alpha))
        for i in range(len(alpha)):
            if alpha[(i+rand)%len(alpha)] == 0 or alpha[(i+rand)%len(alpha)] == C:
                continue
            if takeStep((i+rand)%len(alpha),i2):
                return 1
        rand = random.randint(0,len(alpha))
        for i in range(len(alpha)):
            i1 = (i+rand)%len(alpha)
            if takeStep(i1,i2):
                return 1
    return 0

def train():
    numChanged = 0
    examineAll = 1
    loopnum = 0
    totaloop = 100
    while (numChanged > 0 or examineAll):
        numChanged = 0
        if loopnum == totaloop:
           break
        else:
            loopnum += 1
            print(f"\rLoops:{loopnum}/{totaloop}",end="")
        if examineAll:
            for i in range(len(point)):
                numChanged != examineExample(i)
        else:
            for i in range(len(alpha)):
                if alpha[i] != 0 or alpha[i] != C:
                    numChanged += examineExample(i)
        if examineAll == 1:
            examineAll = 1
        elif numChanged == 0:
            examineAll = 1
    print(" Finish!")
    return -1

train()

output = svm_output(alpha, target, kernel, point, point, b)
res = []
for i in output:
    if i > 0:
        res.append(1)
    else:
        res.append(-1)
print(target)
print(res)

print(precision_score(target,res))
print(recall_score(target,res))
print(f1_score(target,res))

/var/folders/4w/x2czcsyx1m58qm0lp_wbk1jm0000gn/T/ipykernel_59108/1085361638.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target[target == 0] = -1


Loops:100/100 Finish!
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -